# Example code for looking up best matching contact sample from PICO-db

In order to reconstruct interaction from a brand new image, we need paired human+object contact annotations and the corresponding object mesh. We can use the human contact vertices as a query key to find the best matching sample in the PICO-db (or any similar) dataset, and apply that to our current interaction image.

First we load the human contacts from PICO-db into a dictionary, per object categories.

The input/query for the looking up the closest sample is the SMPL human contact, since this can be inferred for a new image relatively easily and accurately, using e.g. [DECO](https://github.com/sha2nkt/deco).

Here we define best match in terms of intersection over union (IoU) between the inferred human contact and human contact samples in the loaded lookup dataset, of course within the respective object category.

We can just take the top result (contact mapping + object mesh) and apply it to the new image, or we can also consider the object crop-to-mesh match for the top-k results using a tool like [https://github.com/saidwivedi/Object_Retrieval](https://github.com/saidwivedi/Object_Retrieval) and make a joint decision based on both scores.

In [38]:
import os
import json
from tqdm import tqdm


def get_human_verts_from_mapping(contact_transfer_map_path):
    contact_mapping_json = json.load(open(contact_transfer_map_path, 'r'))
    
    # simplify the dictionary
    contact_transfer_map = {}
    for elem in contact_mapping_json['data']:
        contact_transfer_map[elem['name']] = elem['contactPoints']
    
    # delete all keys that don't start with 'human'
    keys_to_delete = [key for key in contact_transfer_map if not key.startswith('human')]
    for key in keys_to_delete:
        contact_transfer_map.pop(key)

    # make a flat list of all human contact vertices
    human_contact_verts = []
    for key in contact_transfer_map:
        human_contact_verts += contact_transfer_map[key]
    human_contact_verts = list(set(human_contact_verts))
    human_contact_verts = [int(x[2:]) for x in human_contact_verts]

    return human_contact_verts


def load_dataset(annotations_path):
    if os.path.exists('picodb_human_contacts.json'):
        with open('picodb_human_contacts.json', 'r') as f:
            picodb_human_contacts = json.load(f)

    else:
        picodb_human_contacts = {}
        for ann_f in tqdm(os.listdir(annotations_path)):
            obj_category = ann_f.split('__')[0]
            human_contact_verts = get_human_verts_from_mapping(os.path.join(annotations_path, ann_f))

            if obj_category not in picodb_human_contacts:
                picodb_human_contacts[obj_category] = []
            picodb_human_contacts[obj_category].append((ann_f, human_contact_verts))

        # save dataset
        with open('picodb_human_contacts.json', 'w') as f:
            json.dump(picodb_human_contacts, f)

    print('Number of samples loaded:', len([item for sublist in picodb_human_contacts.values() for item in sublist]))
    return picodb_human_contacts


def find_closest_human_contact(picodb_human_contacts, obj_category, contact_verts, nr_results=3):
    ious = []
    for imgf, human_contact_verts in picodb_human_contacts[obj_category]:
        iou = len(set(contact_verts) & set(human_contact_verts)) / len(set(contact_verts) | set(human_contact_verts))
        ious.append((imgf, iou))

    ious = sorted(ious, key=lambda x: x[1], reverse=True)
    return ious[:nr_results]


In [ ]:
# in order to reconstruct interaction from a brand new image,
# we need paired human+object contact annotations and the corresponding object mesh
# we can use the human contact vertices as a query key to find the best matching sample in the dataset
# and apply that to the current image

# load a contact dataset (here: picodb) for lookup
annotations_path = '../../data/annotations'
picodb_human_contacts = load_dataset(annotations_path)

# input: SMPL human contact verts to find closest contact in the dataset for
# can be obtained for a new image using e.g. DECO (https://github.com/sha2nkt/deco)
current_human_contact = list(range(2000, 2700)) # example contact verts

# find the closest matches (IoU) within an object category
closest_human_contacts = find_closest_human_contact(
    picodb_human_contacts = picodb_human_contacts,
    obj_category = 'baseball_bat',
    contact_verts = current_human_contact,
    nr_results = 3
)
print('\nClosest human contacts:')
for imgf, iou in closest_human_contacts:
    print(f'Found match: {imgf} with IoU: {iou}')

# we can just take the top result (contact mapping + object mesh) and apply it to the new image
# or we can also consider the object crop-to-mesh match for the top k results
# using a tool like https://github.com/saidwivedi/Object_Retrieval
# and make a joint decision based on both scores

Number of samples loaded: 4123

Closest human contacts:
Found match: baseball_bat__vcoco_000000404613.json with IoU: 0.43889618922470436
Found match: baseball_bat__hake_train2015_HICO_train2015_00005083.json with IoU: 0.31565440547476475
Found match: baseball_bat__hake_train2015_HICO_train2015_00006027.json with IoU: 0.31276778063410454
